# Data Clean up
Updated on: 2022-08-10 10:20:48 CEST

Author: Abzer Kelminal (abzer.shah@uni-tuebingen.de) <br>
Input file format: .csv files or .txt files <br>
Outputs: .csv files  <br>
Dependencies: ggplot2, dplyr, IRdisplay

<p style='text-align: justify;'>This Notebook is used for cleaning the feature table, an output of metabolomics experiment, containing all the MS/MS features (or peaks) with their corresponding intensities. The data cleanup steps involved are: 1) Blank removal 2) Imputation 3) Normalisation.</p>

<div class="alert alert-block alert-warning">
    
Kindly download this Jupyter Notebook and run it in your local computer for successful running of the code. To know more about how to get the Jupyter Notebook running with R code, please have a look at these documents: [GitHub Link for installing Jupyter using Anaconda](https://github.com/Functional-Metabolomics-Lab/Jupyter-Notebook-Installation/blob/main/Anaconda%20with%20R%20kernel%20installation.pdf), [For directly installing Jupyter without Anaconda](https://simply-python.com/2019/06/24/running-r-on-jupyter-notebook-with-r-kernel-no-anaconda/) 
    
This Notebook can also be opened and followed in <b>Google Colab.</b> Further details on using Google Collab and giving your input files are provided in later sections. 
    
</br>    
<b><u><font size=3>SPECIAL NOTE:</u> </b>

- <b><font size=3> Please read the comments before proceeding with the code and let us know if you run into any errors and if you think it could be commented better. We would highly appreciate your suggestions and comments!! </font> </b>
- <b><font size=3> Also please look for any updates in our GitHub page: [Functional-Metabolomics-Lab](https://github.com/Functional-Metabolomics-Lab/Data_Cleanup) </font></b>
</div>

# Brief introduction about the Test data:
<div class="alert alert-block alert-success"> 
    
- <p style='text-align: justify;'> Bacteria (B.subtilis and E.coli) were treated with a pool of antibiotics (Sulfamethoxazole, sulfadimethoxine, cyproconazole) and 
    also a xenobiotic, Asulam, taken at a concentration lower than their MIC (minimum inhibitory concentration). </p>
- <p style='text-align: justify;'> The samples were collected at different timepoints, the compounds were extracted (with EtOAc) and measured using LC-MS/MS. </p>
- <p style='text-align: justify;'> The goal of the experiment was to look for any potential biotransformation that happened with increase in time. eg: Drug or xenobiotic metabolism </p>
</div>

# Package installation:

In [ ]:
#installing and calling the necessary packages:
if (!require("ggplot2")) install.packages("ggplot2")
if (!require("dplyr")) install.packages("dplyr")
if (!require("IRdisplay")) install.packages("IRdisplay")

library(ggplot2) # For plots
library(dplyr) # For data manipulation
library(IRdisplay) # For displaying outputs

In [ ]:
#'Global' settings for plot size in the output cell
#options(repr.plot.width=10, repr.plot.height=8,res=600) #For google collab
options(repr.plot.width=5, repr.plot.height=3) #For Jupyter Notebook

# Input files needed for the Notebook:
1) <b>Feature table:</b> An output of metabolomics experiment, containing all the features or peaks (LC-MS/MS peaks here) with their corresponding intensities. The feature table used in the test data is obtained by MZmine. </br>
    -   Gap-filled (A typical output of MZmine) </br>
    -   Non gapfilled (You can still run the script without non-gapfilled feature file) </br>
2) <b>Metadata:</b> Created by the user about the files used obtaining the feature table. The columns in a metadata should be created with the following format: filename (1st column having all the filenames in the same order as the columns in feature table), all the other columns with column name such as: ATTRIBUTE_yourDesiredAttribute. </br>

Please have a look at the metadata used here for reference. Creating a metadata in the above-mentioned format is necessary for uploading the files in GNPS and to obtain a molecular network.

# Getting the input files:

## 1) Reading the input data using URL (from GitHub):
Here, we can directly pull the data files from our Functional Metabolomics Github page:

In [ ]:
## Non-gap filled
nft_url <- 'https://raw.githubusercontent.com/abzer005/Data_Cleanup/main/MZmine%20Files/20220726_Xenobiotic_metabolism_Nongapfilled_quant.csv'
## Gap filled
ft_url <- 'https://raw.githubusercontent.com/abzer005/Data_Cleanup/main/MZmine%20Files/20220726_Xenobiotic_metabolism_Gapfilled_quant.csv'
md_url <- 'https://raw.githubusercontent.com/abzer005/Data_Cleanup/main/MZmine%20Files/20220726_Xenobiotic_Metabolism_metadata.txt'

In [ ]:
nft <- read.csv(nft_url, header = T, check.names = F)
ft <- read.csv(ft_url, header = T, check.names = F)
md <- read.csv(md_url, header = T, check.names = F, sep = '\t')

## 2) Setting a local working directory and creating an automatic result directory:

### 2.1 For direct Jupyter Notebook Users:
<p style='text-align: justify;'> <font color='green'> <b> Setting a local working directory with Jupyter Notebook is very easy.</b> You can simply copy the path of the folder containing all your input files in your local computer to the output line of the next cell. It will be set as your working directory (or working folder)  </font></p> 
For ex: D:\User\Project\Test_Data <br>

In [ ]:
# setting the current directory as the working directory
Directory <- normalizePath(readline("Enter the path of the folder with input files: "),"/",mustWork=FALSE)
setwd(Directory)

In [ ]:
getwd() #to get the working directory 

### 2.2 For Google Colab Users:
<p style='text-align: justify;'> <font color='red'>For Google Colab, it is not possible to access the files from your local computer as it is hosted on Google's cloud server. An easier workaround is to upload the necessary files into the Google colab session using the 'Files' icon on the left as shown in the image. The code in the next cell creates a new folder 'My_TestData' in the Colab space and sets the folder as working directory. Following the steps in the image, you can check in your Colab to see if the folder has been created. Once you see it, simply upload the files from your local PC to the folder 'My_TestData' and then continue running the rest of the script.</font> </p>

<p style='text-align: justify;'>SPECIAL NOTE: All the files uploaded to Google Colab would generally disappear after 12 hours. Similarly, all the outputs would be saved only in the Colab, so we need to download them into our local system at the end of our session.</p> 

[Go to section: Getting outputs from Colab](#colab_output) 

In [ ]:
#Only for Google Colab:
dir.create("/content/My_TestData", showWarnings = TRUE, recursive = FALSE, mode = "0777")
setwd("/content/My_TestData")

![Google-Colab Files Upload](https://github.com/abzer005/Images-for-Jupyter-Notebooks/blob/main/StepsAll.png?raw=true)

In [ ]:
# Getting all the files in the folder
dirs <- dir(path=paste(getwd(), sep=""), full.names=TRUE, recursive=TRUE)
folders <- unique(dirname(dirs))
files <- list.files(folders, full.names=TRUE)
files_1 <- basename((files))
files_2 <- dirname((files))
# Creating a Result folder
dir.create(path=paste(files_2[[1]], "_Results", sep=""), showWarnings = TRUE)
fName <-paste(files_2[[1]], "_Results", sep="")

IRdisplay::display(data.frame(INDEX=c(1:length(files_1)),Filename=files_1))

In the following line, enter the required file index numbers separated by commas. For example, in our case, as: 1,3. The accepted file formats are csv,txt and tsv.

In [ ]:
input <- as.double(unlist(strsplit(readline("Specify the file index of gapfilled feature-file, metadata separated by commas:"), split=",")))

#Gets the extension of each file. Ex:csv
pattern <- c()
  for (i in files_1){
    sep_file <- substr(i, nchar(i)-2,nchar(i))
    pattern <- rbind(pattern,sep_file)
  }
#pattern

ft <- read.csv(files_1[input[1]],sep = ifelse(pattern[input[1]]!="csv","\t",","), header=TRUE,check.names = FALSE) # By adding 'row.names = 1' in the code, the 1st column 'ID' becomes the row names
md <-read.csv(files_1[input[2]], sep = ifelse(pattern[input[2]]!="csv","\t",","), header=TRUE,check.names = FALSE)

<p style='text-align: justify;'> Since non gapfilled feature file is not a direct output of mzMine, not everybody will have it. Therefore, we have the next cell separately. For the imputation step, we can also impute the feature table without a non-gapfilled feature file.</p>

 More about this in the later section: [Imputation with a Cutoff LOD](#Impute_LOD)

In [ ]:
#If you have non gapfilled feature file:
read_cmd <- readline("Do you have non gap-filled feature table? Y/N:")
if(read_cmd=="Y"){
    x <- as.double(readline("Enter the ID number of non-gap-filled feature file:"))
    nft<- read.csv(files_1[x],sep=ifelse(pattern[x]!="csv","\t",","), header = TRUE,check.names = FALSE)
}else if(read_cmd != "N"){print("Please enter either Y or N")}

Lets check if the data has been read correclty!!

In [ ]:
head(ft)
dim(ft)

In [ ]:
head(nft)
dim(nft)

In [ ]:
head(md)
dim(md)

<p style='text-align: justify;'> In the next cell, we are trying to bring the feature table and metadata in the correct format such as the rownames of metadata and column names of feature table are the same. They both are the file names and they need to be the same, as from now on, we will call the columns in our feature table based on our metadata information. Thus, using the metadata, the user can filter their data easily. You can also directly deal with your feature table without metadata by getting your hands dirty with some coding!! But having a metadata improves the user-experience greatly. </p>

In [ ]:
#Removing Peak area extensions
colnames(ft) <- gsub(' Peak area','',colnames(ft))
md$filename<- gsub(' Peak area','',md$filename)

#Removing if any NA columns present in the md file
md <- md[,colSums(is.na(md))<nrow(md)]

#Changing the row names of the files
rownames(md) <- md$filename
md <- md[,-1]
rownames(ft) <- paste(ft$'row ID',round(ft$'row m/z',digits = 3),round(ft$'row retention time',digits = 3), sep = '_')

#Picking only the files with column names containing 'mzML'
ft <- ft[,grep('mzML',colnames(ft))]

# if nft file exists, we perform all the above for nft as well
if(exists("nft")==T){
    colnames(nft) <- gsub(' Peak area','',colnames(nft))
    rownames(nft) <- paste(nft$'row ID',round(nft$'row m/z',digits = 3),round(nft$'row retention time',digits = 3), sep = '_')
    nft <- nft[,grep('mzML',colnames(nft))]
}

In [ ]:
new_ft<- ft[,order(colnames(ft))] #ordering the ft by its column names
new_md <-md[order(rownames(md)),] #ordering the md by its row names

#lists the colnames(ft) which are not present in md
unmatched_ft <- colnames(new_ft)[which(is.na(match(colnames(new_ft),rownames(new_md))))] 
cat("These", length(unmatched_ft),"columns of feature table are not present in metadata:")
if((length(unmatched_ft) %% 2) ==0)
{matrix(data=unmatched_ft,nrow=length(unmatched_ft)/2,ncol=2)}else
{matrix(data=unmatched_ft,nrow=(length(unmatched_ft)+1)/2,ncol=2)}

flush.console()
Sys.sleep(0.2)

#lists the rownames of md which are not present in ft
unmatched_md <- rownames(new_md)[which(is.na(match(rownames(new_md),colnames(new_ft))))] 
cat("These", length(unmatched_md),"rows of metadata are not present in feature table:")
if((length(unmatched_md) %% 2) ==0){
    matrix(data=unmatched_md,nrow=length(unmatched_md)/2,ncol=2)
}else{
    matrix(data=unmatched_md,nrow=(length(unmatched_md)+1)/2,ncol=2)}

#Removing those unmatching columns and rows:
if(length(unmatched_ft)!=0){new_ft <- subset(ft, select = -c(which(is.na(match(colnames(ft),rownames(md))))) )}
if(length(unmatched_md)!=0){new_md <- md[-c(which(is.na(match(rownames(md),colnames(ft))))),]}

In [ ]:
#checking the dimensions of our new ft and md:
cat("The number of rows and columns in our original ft is:",dim(ft),"\n")
cat("The number of rows and columns in our new ft is:",dim(new_ft),"\n")
cat("The number of rows and columns in our new md is:",dim(new_md))

Notice that the number of columns of feature table is same as the number of rows in our metadata. Now, we have both our feature table and metadata in the same order.

In [ ]:
new_ft<- new_ft[,order(colnames(new_ft))] #ordering the ft by its column names
new_md <-new_md[order(rownames(new_md)),] #ordering the md by its row names
#checking if they are the same
if(identical(colnames(new_ft),rownames(new_md))==T)
   {print("The column names of ft and rownames of md are the same")}else{print("The column names of ft and rownames of md are NOT THE SAME!!")}

Lets check the files once again!!

In [ ]:
head(nft)
dim(nft)

In [ ]:
head(new_ft)
dim(new_ft)

In [ ]:
head(new_md)
dim(new_md)

# Creating Functions:
<p style='text-align: justify;'> Before getting into the Data cleanup steps, we have created some functions that can be used later for subsetting and plotting. By creating functions, we don't have to write all these big codes multiple times. Instead, we just use the function name. <font color="red">The following cells in this section will not produce any outputs here. </font> The outputs will be produced when we give input variables to these functions in the later sections. </p>


1. <b>Function InsideLevels:</b></br>
Using this function, we get an idea of the multiple levels in each of the metioned attributes in the metadata as well as the datatype of each attribute.  <font color ="blue"> This function takes metadata table as its input. </font>

In [ ]:
#Function: InsideLevels
 InsideLevels <- function(metatable){
    lev <- c()
    typ<-c()
    for(i in 1:ncol(metatable)){
      x <- levels(droplevels(as.factor(metatable[,i])))
      if(is.double(metatable[,i])==T){x=round(as.double(x),2)}
      x <-toString(x)
      lev <- rbind(lev,x)
      
      y <- class(metatable[,i])
      typ <- rbind(typ,y)
    }
    out <- data.frame(INDEX=c(1:ncol(metatable)),ATTRIBUTES=colnames(metatable),LEVELS=lev,TYPE=typ,row.names=NULL)
    return(out)
  }

2. <b>Function SubsetLevels:</b></br>
Using this function, we can subset a dataframe using multiple attributes at a time as defined by the user. In addition to that, we can mention whether we want to keep or exclude certain levels within each selected attribute. <font color ="blue"> It takes metadata table as its input. </font>

In [ ]:
#Function: SubsetLevels  
SubsetLevels <- function(metatable){
    IRdisplay::display(InsideLevels(metatable)) # use show() when working in RStudio
    Condition <- as.double(unlist(strsplit(readline("Enter the IDs of interested attributes to subset (separaed by commas if more than one attribute):"), split=',')))
    
    for( i in 1:length(Condition)){
      list_final <- c() 
      #Shows the different levels within each selected condition:
      Levels_Cdtn <- levels(droplevels(as.factor(metatable[,Condition[i]])))
      subset_meta <- data.frame(Index=1:length(Levels_Cdtn),Levels_Cdtn)
      colnames(subset_meta)[2] <- paste("Levels_",colnames(metatable[Condition[i]]))
      IRdisplay::display(subset_meta) # use show() when working in RStudio
      
      
      #Among the shown levels of an attribute, select the ones to keep or exclude:
      Read_cdtn <- readline("Do you want to keep or exclude few conditions? K/E: ")
      if( Read_cdtn=="K"){
        temp <- as.double(unlist(strsplit(readline("Enter the index numbers of condition(s) you want to KEEP (separated by commas):"), split=',')))
        ty <- class(metatable[,Condition[i]])
        list_keep <-Levels_Cdtn[temp, drop=F]
        list_exc <-Levels_Cdtn[-temp, drop=F]
        cat("The condition(s) you want to exclude in ",colnames(metatable)[Condition[i]]," :",list_exc,"\n")
        cat("The condition(s) you want to keep in ",colnames(metatable)[Condition[i]]," :",list_keep,"\n")
        
        }else if(Read_cdtn=="E"){
          temp <- as.double(unlist(strsplit(readline("Enter the index numbers of condition(s) you want to EXCLUDE (separated by commas):"), split=',')))
          ty <- class(metatable[,Condition[i]])
          list_exc <-Levels_Cdtn[temp, drop=F]
          list_keep <-Levels_Cdtn[-temp, drop=F]
          cat("The condition(s) you want to exclude in ",colnames(metatable)[Condition[i]]," :",list_exc,"\n")
          cat("The condition(s) you want to keep in ",colnames(metatable)[Condition[i]]," :",list_keep,"\n")
        
          }else{
            print("Sorry! You have given a wrong input!! Please enter either K or E")
            break
      }
      
      #In order to keep the original datatype of the columns, we define the following conditions, else it would all become 'characters or factors'
      if(ty=="integer"){list_keep <- as.integer(list_keep);list_exc <- as.integer(list_exc)
      }else if(ty=="double"){list_keep <- as.double(list_keep);list_exc <- as.double(list_exc)
      }else if(ty=="numeric"){list_keep <- as.numeric(list_keep);list_exc <- as.numeric(list_exc)}
      
      #Gets all the elements in list_keep into list_final 
      for(j in 1:length(list_keep)){
        sub_list <- metatable[(metatable[,Condition[i]] == list_keep[j]),]
        list_final <- rbind(list_final,sub_list)
      }
      metatable <- list_final #list_final again called as metatable in order to keep it in the for-loop for further subsetting
    }
    return(metatable)
  }

3. <b>Function FrequencyPlot:</b></br>
<p style='text-align: justify;'><font color ="blue">The below function takes two input datatables: for example, gapfilled and non-gapfilled as its inputs</font>, calculates the frequency distribution of the data in the order of 10 and produces a grouped barplot showing the distribution as output. The frequency plot shows where the features are present in higher number.</p>

In [ ]:
#Function: FrequencyPlot
FrequencyPlot <- function(x1,x2){
  
   #creating bins from -1 to 10^10 using sequence function seq()
    bins <- c(-1,0,(1 * 10^(seq(0,10,1)))) 
    
    #cut function cuts the give table into its appropriate bins
    scores_x1 <- cut(as.matrix(x1),bins,labels = c('0','1','10','1E2','1E3','1E4','1E5','1E6','1E7','1E8','1E9','1E10')) 
    
    #transform function convert the tables into a column format: easy for visualization 
    Table_x1<-transform(table(scores_x1)) #contains 2 columns: "scores_x1", "Freq"
    
    #Repeating the same steps for x2
    scores_x2 <- cut(as.matrix(x2),bins,labels = c('0','1','10','1E2','1E3','1E4','1E5','1E6','1E7','1E8','1E9','1E10'))
    Table_x2 <- transform(table(scores_x2))
  
    #Getting the names of x1 and x2
    arg1 <- deparse(substitute(x1))
    arg2 <- deparse(substitute(x2))
    
    #Creating a data frame for plotting
    data_plot <- as.data.frame(c(Table_x1$Freq,Table_x2$Freq)) #Concatenating the frequency info of both tables rowwise
    colnames(data_plot) <- "Freq" #naming the 1st column as 'Freq'
    data_plot$Condition <- c(rep(arg1,12),rep(arg2,12)) #adding a 2nd column 'Condition', which just repeats the name of x1 and x2 accordingly
    data_plot$Range_bins <- rep(Table_x1$scores_x1,2) #Adding 3rd column 'Range Bins'
    data_plot$Log_Freq <- log(data_plot$Freq+1) #Log scaling the frequency values
    
    ## GGPLOT2
    BarPlot <- ggplot(data_plot, aes(Range_bins, Log_Freq, fill = Condition)) + 
    geom_bar(stat="identity", position = "dodge", width=0.4) + 
    scale_fill_brewer(palette = "Set1") +
    ggtitle(label="Frequency plot") +
    xlab("Range") + ylab("(Log)Frequency") + labs(fill = "Data Type") + 
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) +   # setting the angle for the x label
    theme(axis.text.y = element_text(angle = 45, vjust = 0.5, hjust=1)) +   # setting the angle for the y label
    theme(plot.title = element_text(hjust = 0.5)) # centering the plot title
  
    print(BarPlot)
}  

# Splitting the data into Blanks and Samples using Metadata:
<a id="data_split"></a>

For the first step: Blank removal, we need to split the data as spectra obtained from blanks and samples respectively using the metadata. More about Blank removal in the next section.

<p style='text-align:justify;'> Before doing anything, let's have a quick look at the metadata by running the next cell. It gives us the information of the multiple levels within each attribute in the metadata. Also check the dimensions to see the number of rows and columns in the metadata. The number of files for B.sub = 40, E.coli = 40, Media =2, hence in total, 82 rows and 6 columns of attributes. </p>

In [ ]:
InsideLevels(new_md)
dim(new_md)

### Subsetting the dataset:
<p style='text-align:justify;'> When having several groups in the data, as in our case, different species like B.sub and E.coli, we can subset the data accordingly using the next cell. For example: We can split it as B.sub + Media and/or E.coli + Media. We need to keep the media here as these are the blanks and their features need to be removed from that of the samples. This would be done in the Blank removal step.</p>

<p style='text-align:justify;'> <b>For subsetting:</b> The users can choose the first attribute and <b>keep B.sub and Media</b> (or) simply <b>exclude E.coli.</b> You can also repeat the steps later with E.coli & Media. You can also not subset anything here by skipping the next 2 cells and move on with the code for splitting the data into Samples and blanks. </p>

In [ ]:
subset_data <- SubsetLevels(new_md)

Lets see if it has been subsetted properly! Also check the dimensions to see if the number of rows in the metadata has been reduced.

In [ ]:
InsideLevels(subset_data)
dim(subset_data)

Once we subset the data according to a group, we can further proceed to split the blanks from the sample in the cell below. As mentioned before, if no subsetting is involved, you can simply split your metadata into blank and sample.

In [ ]:
#If subset_data exists, it will take it as "data", else take Meta_filter as "data"
if(exists("subset_data")==T){data <-subset_data}else{data <-new_md}

Condition <- as.double(unlist(readline("Enter the index number of the attribute to split sample and blank:")))
Levels_Cdtn <- levels(droplevels(as.factor(data[,Condition[1]])))
IRdisplay::display(data.frame(INDEX=c(1:length(Levels_Cdtn)),LEVELS=Levels_Cdtn))
 
#Among the shown levels of an attribute, select the ones to keep
Blk_id <- as.double(unlist(readline("Enter the index number of your BLANK:")))
paste0('You chosen blank is:',Levels_Cdtn[Blk_id])
  
#Splitting the data into blanks and samples based on the metadata
md_Blank <- data[(data[,Condition] == Levels_Cdtn[Blk_id]),]
Blank <- new_ft[,which(colnames(new_ft)%in%rownames(md_Blank)),drop=F] 
md_Samples <- data[(data[,Condition] != Levels_Cdtn[Blk_id]),]
Samples <- new_ft[,which(colnames(new_ft)%in%rownames(md_Samples)),drop=F] 

In [ ]:
head(Blank)
dim(Blank)

In [ ]:
head(Samples)
dim(Samples)

**Now that we have our data ready, we can start with the cleanup steps!!**

# Step1: Blank Removal

<p style='text-align: justify;'> In LC-MS/MS, we use solvents called Blanks which are usually injected time-to-time to prevent carryover of the sample. The features coming from these Blanks would also be detected by LC-MS/MS instrument. Our goal here is to remove these features from our samples. The other blanks that can be removed are: Signals coming from growth media alone in terms of microbial growth experiment, signals from the solvent used for extraction methods and so on. Therefore, it is best practice to measure mass spectra of these blanks as well in addition to your sample spectra. </p>

**How do we remove these blank features?** </br> 
<p style='text-align: justify;'> Since we have the feature table split into Control blanks and Sample groups now, we can compare blanks to the sample to identify the background features coming from blanks. A common filtering method is to use a cutoff to remove features that are not present sufficient enough in our biological samples. </p>

The steps followed in the next few cells are:
1. <p style='text-align: justify;'> We find an average for all the feature intensities in your blank set and sample set. Therefore, for n no.of features in a blank or sample set, we get n no.of averaged features. </p>
2. <p style='text-align: justify;'> Next, we get a ratio of this average_blanks vs average_sample. This ratio Blank/sample tells us how much of that particular feature of a sample gets its contribution from blanks. If it is more than 30% (or Cutoff as 0.3), we consider the feature as noise. </p>
3. <p style='text-align: justify;'> The resultant information (if ratio > Cutoff or not) is stored in a bin such as 1 = Noise or background signal, 0 = Feature Signal</p>
4. <p style='text-align: justify;'> We count the no.of features in the bin that satisfies the condition ratio > cutoff, and consider those features as 'noise or background features' and remove them. </p>

For a dataset containing several groups, the filtering steps are performed group-wise using the previous section [Subsetting the dataset](#data_split)

**<font color='red'> The Cutoff used to obtain the all the files in MZmine Results folder is 0.3 </font>**

In [ ]:
if(readline('Do you want to perform Blank Removal- Y/N:')=='Y'){
    
    #When cutoff is low, more noise (or background) detected; With higher cutoff, less background detected, thus more features observed
    Cutoff <- as.numeric(readline('Enter Cutoff value between 0.1 & 1:')) # (i.e. 10% - 100%). Ideal cutoff range: 0.1-0.3
    
    #Getting mean for every feature in blank and Samples
    Avg_blank <- rowMeans(Blank, na.rm= FALSE, dims = 1) # set na.rm = FALSE to check if there are NA values. When set as TRUE, NA values are changed to 0
    Avg_samples <- rowMeans(Samples, na.rm= FALSE, dims = 1)
    
    #Getting the ratio of blank vs Sample
    Ratio_blank_Sample <- (Avg_blank+1)/(Avg_samples+1)
    
    # Creating a bin with 1s when the ratio>Cutoff, else put 0s
    Bg_bin <- ifelse(Ratio_blank_Sample > Cutoff, 1, 0 )
    Blank_removal <- cbind(Samples,Bg_bin)

    # Checking if there are any NA values present. Having NA values in the 4 variables will affect the final dataset to be created
    temp_NA_Count <-cbind(Avg_blank ,Avg_samples,Ratio_blank_Sample,Bg_bin)
    
    print('No of NA values in the following columns:')
    print(colSums(is.na(temp_NA_Count)))

     #Calculating the number of background features and features present
    print(paste("No.of Background or noise features:",sum(Bg_bin ==1,na.rm = TRUE)))
    print(paste("No.of features after excluding noise:",(nrow(Samples) - sum(Bg_bin ==1,na.rm = TRUE)))) 

    Blank_removal <- Blank_removal %>% filter(Bg_bin == 0) # Taking only the feature signals
    Blank_removal <- as.matrix(Blank_removal[,-ncol(Blank_removal)]) # removing the last column Bg_bin 
}

In [ ]:
write.csv(Blank_removal, file.path(fName,paste0('Blank_removed.csv')),row.names =TRUE) 

In [ ]:
head(Blank_removal)
dim(Blank_removal)

# Step 2: Imputation

<p style='text-align: justify;'> For several reasons, real world datasets might have some missing values in it, in the form of NA, NANs or 0s. Eventhough the gapfilling step of MZmine fills the missing values, we still end up with some missing values or 0s in our feature table. This could be problematic for statistical analysis. </p> 
<p style='text-align: justify;'> In order to have a better dataset, we cannot simply discard those rows or columns with missing values as we will lose a chunk of our valuable data. Instead we can try imputing those missing values. Imputation involves replacing the missing values in the data with a meaningful, reasonable guess. There are several methods, such as: </p> 
  
1) Mean imputation (replacing the missing values in a column with the mean or average of the column)  
2) Replacing it with the most frequent value  
3) Several other machine learning imputation methods such as k-nearest neighbors algorithm(k-NN), Hidden Markov Model(HMM)

## 1) Imputation using Non-gapfilled feature table:
One such method, we are going to use is: **to replace the zeros from the gapfilled quant table with the non-gap filled table** we get from MZmine. In order to do that, we can visualize our data distribution using the frequenct plot

In [ ]:
GapFilled <- Blank_removal
if(exists("nft")==T){NotGapFilled <- nft}

In [ ]:
if(readline('Do you want to perform Imputation with minimum value of NonGapFilled table? - Y/N:')=='Y'){
    plot<- FrequencyPlot(GapFilled,NotGapFilled)
    
    Arg1 = plot$data$Condition[1]
    Arg2 = plot$data$Condition[13]
    
    # getting the 2nd minimum value of non-gap filled data. (The first minimum value in the data table is usually zero)
    RawLOD <- round(min(NotGapFilled[NotGapFilled!=min(NotGapFilled)]))
    
    print(paste0("The minimum value greater than 0 for ",Arg1,":", round(min(GapFilled[GapFilled!=min(GapFilled)]))))
    print(paste0("The minimum value greater than 0 for ",Arg2,":", RawLOD))

    Imputed <- GapFilled
    Imputed[Imputed<RawLOD] <- RawLOD # Replacing values<RawLOD with RawLOD
    head(Imputed) 
}

In [ ]:
write.csv(Imputed, file.path(fName,paste0('Imputed_QuantTable_filled_with_MinValue_',RawLOD,'_CutOff_Used_',Cutoff,'.csv')),row.names =TRUE) 

## 2) Imputation with a Cutoff LOD:
<a id="Impute_LOD"></a>

<p style='text-align: justify;'>Instead of replacing with the minimum value of nft, we can also only use ft and see the frquency distribution of its features with the frequency plot. The frequency plot shows where the features are present in higher number.</p>

<p style='text-align: justify;'>For ex: If until range 10-100, (shown in the figure as 1E2) there are no or very less features, we want to exclude until that range and consider from range (100-1000), or, in other words, '1E3' or '1000' as Cutoff_LOD. This value will be used to replace the zeros as well as all the values lower than Cutoff_LOD in the data table.</p>

In [ ]:
FrequencyPlot(GapFilled,GapFilled)

However, if you are not satisfied with the value as shown in the plot, you can enter any value for imputation. For ex: The minimum value is GapFilled table, which in our case is 1175

In [ ]:
if(readline('Do you want to perform Imputation with a Cutoff LOD? - Y/N:')=='Y'){
    Cutoff_LOD <- as.numeric(readline("Enter your Cutoff LOD as seen in the plot or your desired value for imputation:"))  #Enter the LOD value as seen in the frequency plot
    Imputed <- GapFilled
    Imputed[Imputed <Cutoff_LOD] <- Cutoff_LOD
    head(Imputed)
}

In [ ]:
dim(Imputed)

In [ ]:
write.csv(Imputed,file.path(fName, paste0('Imputed_QuantTable_filled_with_',Cutoff_LOD,'_CutOff_Used_',Cutoff,'_Bsub','.csv')),row.names =TRUE)

In [ ]:
#removing all the rows with only cutoff values:
#Imputed<-Imputed[rowMeans(Imputed)!= Cutoff_LOD,]  

# Step 3:Normalization
The following code performs sample-centric (column-wise) normalisation:

In [ ]:
if (readline("Do you want to perform Normalization: Y/N:") == 'Y'){
    #Getting column-wise sums of the input-data
    sample_sum <- colSums(Imputed, na.rm= TRUE, dims = 1)
    
    #Dividing each element of a particular column with its column sum
    Normalized_data <- c()
    for (i in 1:ncol(Imputed)){
        x <- Imputed[,i] / sample_sum[i]
        Normalized_data <- cbind(Normalized_data, x)
    }
    colnames(Normalized_data) <- names(sample_sum)
    
    head(Normalized_data)
} else return(head(Imputed))
  
print(paste('No.of NA values in Normalized data:',sum(is.na(Normalized_data)== TRUE)))

In [ ]:
dim(Normalized_data)

In [ ]:
write.csv(Normalized_data, file.path(fName,'Normalised_Quant_table_Bsub_OnlyCutOffsRemoved.csv'),row.names =TRUE) 

All the result files would be available in the 'Result' folder within your working directory (i.e the folder path you mentioned in the beginning of the script).

### Getting output files from Google Colab:
<a id="colab_output"></a>
For Google Collab users, we can zip the result folder which contains all the output files using the next cell and download the zip file directly from the folder "/content/My_TestData" into the local system.

In [ ]:
#Only for Google Colab
zip(zipfile = 'TestData_Results', files = "/content/My_TestData_Results/")